In [ ]:
if True:
    import mne
    import os.path as op
    from matplotlib import pyplot as plt
    from MEEGbuddy import MEEGbuddy
    import glob
    import re,os
    from pandas import read_csv,DataFrame
    import numpy as np
    #
    subject = 'Demo'
    # 
    np.random.seed(13)
    #
    d = 'data/Demo-raw.fif'
    b = 'data/Demo-behavior.csv'
    if not (op.isfile(b) and op.isfile(d)):
        raw = mne.io.read_raw_fif(op.join(mne.datasets.sample.data_path(),
                                          'MEG', 'sample', 'sample_audvis_raw.fif'))
        raw.info['bads'] = []
        stimulus_events = mne.find_events(raw, stim_channel='STI 001')
        response_events = mne.find_events(raw, stim_channel='STI 002')
        '''These are really left and right audio events but we will pretend
           because this tool was designed for task analysis with a stimulus 
           and optionally a response'''
        responses = np.random.choice(range(len(stimulus_events)),len(response_events),
                                     replace=False)
        bf = {'Trial':np.arange(len(stimulus_events)),
              'Reward':[np.random.random() for _ in range(len(stimulus_events))],
              'Risk':[np.random.random() for _ in range(len(stimulus_events))],
              'Response Time':[np.random.gamma(2) if i in responses else np.nan 
                    for i in range(len(stimulus_events))]}
        bf['RewardType'] = ['High' if r > 0.5 else 'Low' for r in bf['Reward']]
        bf['RiskType'] = ['High' if r > 0.5 else 'Low' for r in bf['Risk']]
        df = DataFrame(bf)
        '''The MNE example dataset uses the trigger heights to code for 
           events, this works well and if efficient unless you have more
           than one variable or a continuous variable (i.e. trial X had
           risk 0.34 and reward $13.19). We are instead using a csv file
           that associates trials with behavior in general. This is all
           taken care of for you by MEEGbuddy all that has to be done is
           a dictionary has to be made with "conditions" as keys and trial 
           values in a list as entries.'''
        df.to_csv(b)
        raw.save(d,overwrite=True)
    #
    stimuli = {'Cue':['STI 001',-0.5,1]}
    response = ['STI 002',-1.5,1]
    baseline = ['STI 001',-1.1,-0.1]
    #
    df = read_csv(b)
    behavior = {}
    for category in list(df.columns):
        behavior[category] = [df.loc[i,category] for i in range(len(df))]
    exclude = [i for i,beh in enumerate(behavior['Response Time']) if np.isnan(beh)]
    exclude_response = [0,len(behavior['Trial'])-1]
    print('Loading into MEEGbuddy ' + subject)
    data = MEEGbuddy(subject=subject,fdata=d,behavior=behavior,baseline=baseline,
                     stimuli=stimuli,meg=True,eeg=True,response=response,no_response=exclude,
                     task='Demo',exclude_response=exclude_response,
                     subjects_dir = os.path.join(os.getcwd(),'data'))

''' MEEGbuddy works on the basic system of keyword with a few built in:
    preprocessed, ica and ar. The keyword preprocessed is automatically
    applied because on principle, we want to leave a copy of the raw data 
    that we started with completely untouched'''

data.autoMarkBads(overwrite=True) 

''' QC that auto mark bads did a good enough job, we don't want really bad
    components in the data because they could swamp our ICA. It's also a huge
    pain to select out bad channels for many subjects, hence this very simple
    approach (NB: not very sensitive which is a good thing because if there is 
    usable data we want to let autoreject try to get it'''

data.plotRaw(preprocessed=True,overwrite=True) 

''' remove DC offset and very low frequency components for visualization.
    usually people don't analyze this low so we can just apply this to the raw
    data without using a keyword argument to save a new branch as a copy '''

data.filterRaw(preprocessed=True,l_freq=0.1) 

''' we already supplied all the information needed to make epochs'''

data.makeEpochs(preprocessed=True,overwrite=True)

''' filters data for visualization and saves it out as
    a copy so as to keep unfiltered data'''

event = 'Response'
condition = 'RewardType'
data.filterEpochs(event,h_freq=40,l_freq=None,keyword_out='40Hz_low_pass') 

''' compares conditions (High vs Low)
    these can be provided by values but if value argument
    is not passed they are selected by default as all the unique
    conditions as you can see from the plot there is a ton of eyeblink
    contamination that ICA will hopefully remove and if not
    those epochs will be rejected (by autoreject)'''

data.plotEvoked(event,condition=condition,keyword='40Hz_low_pass',
                show=True,detrend=1) 

# precompute ICA
data.findICA(preprocessed=True, n_components=50)

# Quality check ICA (will need to do multiple times if the components are hand-selected) (click on the component name to see topo)
ok = True
while ok:
    data.plotICA(preprocessed=True)
    ok = input('Enter to continue, n to go back')

''' I removed components 0,1,16,18 for magnetometers, 2,3,6,19, for gradiometers
    (hard without HEOG) and 0,1,2,7 for eeg. There are three things that show up at first; 
    a evoked time course of each of the ICA source components for each auxillary channel,
    a topo of all the components and a time course of the ICA components raw.plot()-style
    where the components can be selected out. 
    
    The way I choose has three parts: 
    
    First, I look at the topological distributions, distributions that are front heavy 
    and look like eyes are probably blinks, distributions that have a left-right polarization
    (left red, right blue or vice versa) are probably saccades (although these are sometimes
    tricky because they have a tilt to them) and distributions that are back-heavy (occipital)
    are probably heartbeat (eeg 7 is a good example). I also see global components which 
    look like a mostly all red or all blue topo, these are usually the first one or two 
    and in my experience tend to be blinks or mixes of the artifacts. In general there are
    often one major component for each source (blinks, saccades, heartbeat) and sometimes
    a few what I like to think of as harmonics. It would be a lot easier with another EOG 
    and an ECG to pick these components out.
    
    Then, I look at the time course on the larger plot with all the ICA components
    and each smaller eog/ecg epochs plot to confirm that the topo intuition matches
    the actual time course. I also sometimes add components with abnormal topos by
    clicking on the ICA component time course plot. My general outlook on this is
    that if you saw that your results had the topo-distribution of a components,
    if you would think "I think that's eyeblink, I better go back and redo ICA,"
    I would just save yourself the trouble and remove it now. Just because the component
    is time-locked, however, I don't always remove it if it has a non-typical topo
    because in my opinion that is most likely data (these are usually lower contribution
    components anyway).
    
    Finally, I look at the effect on the eog and ecg epochs. Optimally the lines
    would be very flat due to noise cancelling on average. Depending on how 
    agressive you want to be, you can remove  anything that has any time-locked 
    components which will probably include some brain signal (the blinks originated
    from motor commands in the brain after all) but my general opinion is that this 
    is pretty safe to tend towards aggressively removing components.
    
    These aren't the only correct answers and by changing around components more
    biological artifact could potentially be removed but ICA doesn't always capture
    everything which often means the artifact is not very present and thus not something
    to worry about. The data should definitely be usable from here. '''
    
data.makeEpochs(ica=True,overwrite=True)

''' this overwrites the previous epochs used for visualization
    using keyword_out as a input variable we could save out a
    copy but we don't really need the last epochs made using the
    preprocessed but not ICA-applied data.'''

data.filterEpochs(event,h_freq=40,l_freq=None,keyword_out='40Hz_low_pass')
data.plotEvoked(event,condition=condition,keyword='40Hz_low_pass',
                show=True,detrend=1)

data.markBadChannels(['EEG 015', 'MEG 1332', 'MEG 1711'],ica=True)

''' The data looks much better, but channels a few channels look out of wack in the
    evoked plots (you can click on the plots for the channel names) so we'll mark those
    as bad before autoreject. I like the image plots for this because you can really pick out
    channels that are bad by eye much easier. My general approach on this is anything
    that could dramatically throw off your data should be removed because you would not want
    to take the chance that autoreject would miss it.'''
    
# run autoreject on epochs
for event in data.events:
    data.filterEpochs(event,h_freq=100,l_freq=None) 
    '''It is okay to lowpass filter epochs
       because the filter has good spatial
       resolution, but highpassing would
       require too long of a filter so we do that
       earlier. On principle you should keep
       the most of your data you can keep until 
       you have to remove it, which is why this
       is here'''                                 
    data.markAutoReject(event) 
    ''' This will take a long time but you get even clearer data'''

data.filterEpochs(event,h_freq=40,l_freq=None,ar=True,keyword_out='ar_filtered')
data.plotEvoked(event,condition=condition,ylim={'eeg':[-10,10]},keyword='ar_filtered')
''' I like the image plots because you can really see there is
    a similar pattern for both congruent and incongruent trials,
    it is just much more pronounced for incongruent trials, which
    you can't see as well from the evoked plots (the top two).'''
data.plotAutoReject('Response')